# Based on https://www.youtube.com/watch?v=v68zYyaEmEA
We can try solving squirdle using information theory.
given the smaller search space and the type of things to guess this will be much easier.

Each entry has 5 fields:
- Gen: from 1 to 8. Result of a guess can be More(+), Less(-) or Equal(=) 
- Type 1: categorial. Result of a guess can be Not Correct(X), Different Place(↔) or Equal (=)
- Type 2: categorial. Result of a guess can be Not Correct(X), Different Place(↔) or Equal (=)
- Height: float number. Result of a guess can be More(+), Less(-) or Equal(=) 
- Weight: float number. Result of a guess can be More(+), Less(-) or Equal(=) 

But differently, every result is as likely as others ( meaning the probability of the answer being Squirtle is the same as being Charmender ) 
So the "information gain" will just be equal to log(N_AFTER/N_BEFORE) as probability of a certain result being possility is N_AFTER/N_BEFORE.
The entropy will be just the "average" information gain

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
def printWithMarkdown(string:str):
    display(Markdown(string))

def printDfWithMarkdown(df:pd.DataFrame):
    printWithMarkdown(df.to_markdown())

In [2]:
pokedex = pd.read_json("pokedex.json", orient='index')
pokedex.columns=["Gen","Type 1","Type 2", "Height","Weight"]

In [3]:
printDfWithMarkdown(pokedex[(pokedex["Gen"]==3) & (pokedex["Type 1"]=="Normal") &(pokedex["Type 2"]=="")])

|           |   Gen | Type 1   | Type 2   |   Height |   Weight |
|:----------|------:|:---------|:---------|---------:|---------:|
| Zigzagoon |     3 | Normal   |          |      0.4 |     17.5 |
| Linoone   |     3 | Normal   |          |      0.5 |     32.5 |
| Slakoth   |     3 | Normal   |          |      0.8 |     24   |
| Vigoroth  |     3 | Normal   |          |      1.4 |     46.5 |
| Slaking   |     3 | Normal   |          |      2   |    130.5 |
| Whismur   |     3 | Normal   |          |      0.6 |     16.3 |
| Loudred   |     3 | Normal   |          |      1   |     40.5 |
| Exploud   |     3 | Normal   |          |      1.5 |     84   |
| Skitty    |     3 | Normal   |          |      0.6 |     11   |
| Delcatty  |     3 | Normal   |          |      1.1 |     32.6 |
| Spinda    |     3 | Normal   |          |      1.1 |      5   |
| Zangoose  |     3 | Normal   |          |      1.3 |     40.3 |
| Castform  |     3 | Normal   |          |      0.3 |      0.8 |
| Kecleon   |     3 | Normal   |          |      1   |     22   |

### Let's correct some datatypes
Type 1 and Type 2 are categorial data, basically!

In [4]:
pokedex.dtypes

Gen         int64
Type 1     object
Type 2     object
Height    float64
Weight    float64
dtype: object

In [5]:
pokedex["Type 1"] = pokedex["Type 1"].astype("category")
pokedex["Type 2"] = pokedex["Type 2"].astype("category")


### Now lets create a check function
This function returns <, > or = for Gen, Weigth and Height, and O, X or <> for the Type 1 and 2

In [6]:
def compareInt(valA, valB):
    if(valA>valB):
        return "<"
    if(valA<valB):
        return ">"
    return "="
def compareTypeA(valA1, valB1, valB2):
    if(valA1==valB1):
        return "O"
    if(valA1==valB2):
        return "↔"
    return "X"
def check(guess, answer): # Squirtle, Bibarel
    return [compareInt(guess["Gen"],answer["Gen"]), 
        compareTypeA(guess["Type 1"],answer["Type 1"],answer["Type 2"]), 
        compareTypeA(guess["Type 2"],answer["Type 2"],answer["Type 1"]), 
        compareInt(guess["Height"],answer["Height"]), 
        compareInt(guess["Weight"],answer["Weight"])
    ]

In [7]:
print(check(pokedex.loc["Bibarel"], pokedex.loc["Gyarados"]))

['<', 'X', '↔', '>', '>']


### Lets now filter the the dataframe
the numeric can be summarized with a 2-number array, saying the lower and upper bounds
the cathegoric will be lists of the various values

In [8]:
isin = lambda types : np.vectorize(lambda x: x in types)

def filterPokedex(pokedex, gen, type1, type2, height, weight):
    index = pokedex.index
    values = pokedex.values.T

    
    indexes = (
        (values[0]>=gen[0]) & (values[0] <=gen[1]) & 
        isin(type1)(values[1]) & isin(type2)(values[2]) &
        (values[3]>=height[0]) & (values[3]<=height[1]) &
        (values[4]>=weight[0]) & (values[4]<=weight[1]))
    return pokedex.loc[index[indexes]]

In [9]:
filterPokedex(pokedex, [1,2],["Water"],[""],[0,10],[0,100])

,Gen,Type 1,Type 2,Height,Weight
Squirtle,1,Water,,0.5,9.0
Wartortle,1,Water,,1.0,22.5
Blastoise,1,Water,,1.6,85.5
Psyduck,1,Water,,0.8,19.6
Golduck,1,Water,,1.7,76.6
Poliwag,1,Water,,0.6,12.4
Poliwhirl,1,Water,,1.0,20.0
Seel,1,Water,,1.1,90.0
Shellder,1,Water,,0.3,4.0
Krabby,1,Water,,0.4,6.5


### So now we can calculate the Information Gain of a certain guess result. This is only dependant on the number of pokemon before and after the filtering process.
I = -log(Pokemon_After/Pokemon_Before)

In [10]:
def InformationGain(sizeBefore, sizeAfter):
    return - np.log2(sizeAfter/sizeBefore)

### The entropy for a given information is the expected value  probability multiplied by the information gain for each possibility
the probaility of a given outcome for a certain outcome is just  given each result has the same probability, we can now do a function to create the pokedexes needed and calculate the /result for a certain guess

In [11]:
def maxMin(series):
    return [series.values.min(), series.values.max()]

def applyIntFilter(currentBounds, guessValue, result):
    if(result==">"):
        return[guessValue+0.01, currentBounds[1]]
    if(result=="<"):
        return[currentBounds[0], guessValue-0.01]
    return [guessValue, guessValue]

def typesSet(series):
    return set(series.cat.categories)

def filterTypes(types, typeA, typeB, resultA, resultB):
    if(resultA =="O"):
        return {typeA}
    if(resultB == "↔"):
        return {typeB}
    newTypes = types
    if(resultA == "X"):
        newTypes = newTypes-{typeA}
    if(resultB == "X"):
        newTypes = newTypes-{typeB}
    return newTypes

def returnPokeFilter(guess, currentPokedex, result):
    types1 = typesSet(currentPokedex["Type 1"])
    types2 = typesSet(currentPokedex["Type 2"])
    return filterPokedex(currentPokedex,
                         applyIntFilter(maxMin(currentPokedex["Gen"]),guess["Gen"],result[0]),
                         filterTypes(types1,guess["Type 1"],guess["Type 2"], result[1], result[2]),
                         filterTypes(types2,guess["Type 2"],guess["Type 1"], result[2], result[1]),
                         applyIntFilter(maxMin(currentPokedex["Height"]),guess["Height"],result[3]),
                         applyIntFilter(maxMin(currentPokedex["Weight"]),guess["Weight"],result[4]),
                         )

In [12]:
def calculateExpectedEntropyReduction(currentPokedex:pd.DataFrame, guess):
    possible_results = np.array(np.meshgrid(["<",">","="],["O","X","↔"],["O","X","↔"],["<",">","="],["<",">","="], sparse=False)).T.reshape(-1,5)
    entropy = 0
    for result in possible_results:
        if((
                ((result[1]!="↔") | (result[2]!="O")) &
                ((result[2]!="↔") | (result[1]!="O"))
            )):
            # That case is impossible given there is no "Fire, Fire" pokemons
            reducedPokedex = returnPokeFilter(guess,currentPokedex,result)
            if(reducedPokedex.shape[0]!=0):
                entropy = entropy + reducedPokedex.shape[0]/currentPokedex.shape[0]*InformationGain(currentPokedex.shape[0],reducedPokedex.shape[0])
    return entropy

In [13]:
calculateExpectedEntropyReduction(pokedex,pokedex.loc["Alakazam"])

3.8071497201022617

In [14]:
calculateExpectedEntropyReduction(pokedex,pokedex.loc["Bibarel"])

4.393503737338698

### Game itself

Now we can go and create the game itself, with a function calculating which guess has the highest expected information gain

In [15]:
class Squirdle:
    def __init__(self, pokedex, answer):
        self.initial_pokedex = pokedex
        self.current_pokedex = pokedex
        self.answer = answer
    def currentEntropy(self):
        return np.log2(len(self.current_pokedex))
    def guessAName (self, guessName):
        guess = self.initial_pokedex.loc[guessName] # Squirtle
        answer = self.initial_pokedex.loc[self.answer] # Bibarel
        result = check(guess,answer)
        if(guessName==self.answer):
            return "Correct"
        self.apply(guess, result)
        return result
    def apply(self, guess, result):
        self.current_pokedex = returnPokeFilter(guess,self.current_pokedex, result)
        

In [16]:
game = Squirdle(pokedex,"Charizard")

In [17]:
from tqdm.notebook import tqdm
def bestGuess(fullPokedex, currentPokedex):
    bestName = ""
    bestScore = 0
    if(currentPokedex.shape[0]==1):
        bestName = currentPokedex.index[0]
    else:
        pbar = tqdm(fullPokedex.index)
        for pokemon in pbar:
            entropyReduction = calculateExpectedEntropyReduction(currentPokedex, fullPokedex.loc[pokemon])
            if(entropyReduction > bestScore):
                bestScore=entropyReduction
                bestName=pokemon
                pbar.set_postfix({"score":bestScore,"name":bestName})
    print("Best guess is ", bestName," at ", bestScore)
    return bestName
        

For an example play, this is a random one where the answer was "Spiritomb"

In [18]:
game.currentEntropy()

10.036173612553485

In [19]:
bestGuess(game.initial_pokedex, game.current_pokedex)

  0%|          | 0/1050 [00:00<?, ?it/s]

NameError: name 'entropy' is not defined

In [ ]:
game.initial_pokedex

,Gen,Type 1,Type 2,Height,Weight
Bulbasaur,1,Grass,Poison,0.7,6.9
Ivysaur,1,Grass,Poison,1.0,13.0
Venusaur,1,Grass,Poison,2.0,100.0
Mega Venusaur,6,Grass,Poison,2.4,155.5
Charmander,1,Fire,,0.6,8.5
...,...,...,...,...,...
Basculegion,8,Water,Ghost,3.0,110.0
Sneasler,8,Fighting,Poison,1.3,43.0
Overqwil,8,Dark,Poison,2.5,60.5
Enamorus Incarnate Forme,8,Fairy,Flying,1.6,48.0


In [ ]:
game.apply(pokedex.loc["Simipour"],["<","X","X","=",">"])

In [ ]:
game.currentEntropy()

3.321928094887362

In [ ]:
bestGuess(game.initial_pokedex, game.current_pokedex)

Best guess is  Enamorus Therian Forme  at  0.0


'Enamorus Therian Forme'

In [ ]:
game.apply(pokedex.loc["Nidoqueen"],[">","X","X","<",">"])

In [ ]:
game.currentEntropy()

1.0

In [ ]:
game.current_pokedex

,Gen,Type 1,Type 2,Height,Weight
Lunatone,3,Rock,Psychic,1.0,168.0
Spiritomb,4,Ghost,Dark,1.0,108.0


In [ ]:
bestGuess(game.current_pokedex, game.current_pokedex)

Best guess is  Spiritomb  at  0.0


'Spiritomb'

At this point, when there are two possibilities, one can just say whichever. worst - case (like this) will have 2 tries used.</br>
Now we can see the solutions for each problem: This will take a lot. To avoid searching for some solution multiple times we can memorize those in a dict </br>
This dict will have a associate Guess_1:Result_1,Guess_2:Result:2...Guess_n:Result_n to the best guess for that path

In [ ]:
import json
try:
    with open('solutionsFull.json',"r") as file:
        solutions = json.load(file)
except:
    solutions = dict()

In [ ]:
game.current_pokedex

,Gen,Type 1,Type 2,Height,Weight
Venusaur,1,Grass,Poison,2.0,100.0


In [ ]:
performanceInfo = dict()
for pokemon in tqdm(pokedex.index):
    game = Squirdle(pokedex, pokemon)
    path = ""
    result = ""
    tries = 0
    while(result!="Correct"):
        tries+=1
        entropy=game.currentEntropy()
        print("remaining Entropy", entropy)
        if(path in solutions):
            guess = solutions[path]
            print("Applying ", guess, " as it was already found in ",path)
        else:
            if(entropy<=1.5):
                # When entropy is low enough, avoid losing time
                guess = bestGuess(game.current_pokedex, game.current_pokedex)
            else:
                guess = bestGuess(game.initial_pokedex, game.current_pokedex)
            solutions[path] = guess
        result = game.guessAName(guess)
        path += guess+"/"+",".join(result)+"-"
    print("CORRECT")
    performanceInfo = {
        "pokemon":pokemon,
        "path":path,
        "tries":tries
    }
    
    with open('solutionsFull.json',"w") as file:
        json.dump(solutions,file)
    


  0%|          | 0/1050 [00:00<?, ?it/s]

remaining Entropy 10.036173612553485
Applying  Simipour  as it was already found in  
remaining Entropy 6.20945336562895
Applying  Celebi  as it was already found in  Simipour/<,X,X,<,<-
remaining Entropy 1.0
Applying  Bulbasaur  as it was already found in  Simipour/<,X,X,<,<-Celebi/<,X,↔,>,>-
CORRECT
remaining Entropy 10.036173612553485
Applying  Simipour  as it was already found in  
remaining Entropy 2.807354922057604
Applying  Ivysaur  as it was already found in  Simipour/<,X,X,=,<-
CORRECT
remaining Entropy 10.036173612553485
Applying  Simipour  as it was already found in  
remaining Entropy 6.442943495848728


KeyboardInterrupt: 

In [ ]:
game = Squirdle(pokedex, pokemon)

In [ ]:
path

'Simipour/<,X,X,>,>-Claydol/<,X,X,>,<-Charizard/=,X,X,>,>-'

In [ ]:
game.guessAName('Simipour')

['<', 'X', 'X', '>', '>']

In [ ]:
game.current_pokedex

,Gen,Type 1,Type 2,Height,Weight
Venusaur,1,Grass,Poison,2.0,100.0
Charizard,1,Fire,Flying,1.7,90.5
Butterfree,1,Bug,Flying,1.1,32.0
Pidgeotto,1,Normal,Flying,1.1,30.0
Pidgeot,1,Normal,Flying,1.5,39.5
...,...,...,...,...,...
Probopass,4,Rock,Steel,1.4,340.0
Dialga,4,Steel,Dragon,5.4,683.0
Heatran,4,Fire,Steel,1.7,430.0
Giratina Altered Forme,4,Ghost,Dragon,4.5,750.0


In [ ]:
game.guessAName('Claydol')

['<', 'X', 'X', '>', '<']

In [ ]:
game.current_pokedex

,Gen,Type 1,Type 2,Height,Weight
Venusaur,1,Grass,Poison,2.0,100.0
Charizard,1,Fire,Flying,1.7,90.5
Golbat,1,Poison,Flying,1.6,55.0
Dodrio,1,Normal,Flying,1.8,85.2
Aerodactyl,1,Rock,Flying,1.8,59.0
Articuno,1,Ice,Flying,1.7,55.4
Zapdos,1,Electric,Flying,1.6,52.6
Moltres,1,Fire,Flying,2.0,60.0
Noctowl,2,Normal,Flying,1.6,40.8
Crobat,2,Poison,Flying,1.8,75.0


In [ ]:
game.guessAName('Charizard')

['=', 'X', 'X', '>', '>']

In [ ]:
game.current_pokedex

,Gen,Type 1,Type 2,Height,Weight
Venusaur,1,Grass,Poison,2.0,100.0


In [ ]:
bestGuess(game.current_pokedex, game.current_pokedex)

Best guess is    at  0


''